# Machine Learning Notebook - Matt

In [1]:
import HelperFunctions as hf
import numpy as np
import pandas as pd

## Load Data

In [18]:
#Load in the raw data
calendar_df = hf.load_calendar_data()
prices_df = hf.load_sell_price_data()
sales_df = hf.load_sales_train_validation_data()

## Clean Data

In [19]:
#Clean the raw data
df = hf.rawToClean(sales_df, calendar_df, prices_df, days=600, items=100, dropNAPrices=True)

## Feature Engineering

In [20]:
df = hf.rollingMeanDemandFeature(df, windowSize=7, shift=1) #Mean of previous week
df = hf.rollingMeanDemandFeature(df, windowSize=28, shift=1) #Mean of previous 28 days

In [21]:
df = hf.lagFeature(df, var='sold', lag=1) #Amount sold day before
df = hf.lagFeature(df, var='sold', lag=7) #Amount sold a week before
df = hf.lagFeature(df, var='sold', lag=28) #Amount sold 28 days before

In [22]:
df.dropna(inplace=True) #Drop rows with NAs (as result of lagged features)

In [23]:
df.drop(['id'], axis=1, inplace=True) #Drop the id column

In [24]:
df.isnull().sum()

d                    0
sold                 0
wday                 0
month                0
year                 0
snap_CA              0
snap_TX              0
snap_WI              0
sell_price           0
sporting             0
cultural             0
national             0
religious            0
christmas            0
weekend              0
midweek              0
monfri               0
FOODS_1              0
FOODS_2              0
FOODS_3              0
HOBBIES_1            0
HOBBIES_2            0
HOUSEHOLD_1          0
HOUSEHOLD_2          0
FOODS                0
HOBBIES              0
HOUSEHOLD            0
CA_1                 0
CA_2                 0
CA_3                 0
CA_4                 0
TX_1                 0
TX_2                 0
TX_3                 0
WI_1                 0
WI_2                 0
WI_3                 0
CA                   0
TX                   0
WI                   0
rolling_mean_7_1     0
rolling_mean_28_1    0
sold_lag_1           0
sold_lag_7 

In [25]:
df.head(10)

,d,sold,wday,month,year,snap_CA,snap_TX,snap_WI,sell_price,sporting,...,WI_2,WI_3,CA,TX,WI,rolling_mean_7_1,rolling_mean_28_1,sold_lag_1,sold_lag_7,sold_lag_28
2800,29,1,1,2,2011,0,0,0,2.92,0,...,0,0,0,1,0,4.285714,5.214286,4.0,4.0,6.0
2801,29,15,1,2,2011,0,0,0,0.72,0,...,0,0,0,1,0,2.428571,4.285714,0.0,6.0,4.0
2802,29,4,1,2,2011,0,0,0,2.96,0,...,0,0,0,1,0,1.285714,1.678571,1.0,1.0,1.0
2804,29,0,1,2,2011,0,0,0,5.98,0,...,0,1,0,0,1,0.571429,1.142857,1.0,1.0,0.0
2805,29,3,1,2,2011,0,0,0,1.25,0,...,0,0,1,0,0,0.714286,1.464286,1.0,0.0,2.0
2806,29,3,1,2,2011,0,0,0,3.28,0,...,0,0,0,1,0,2.000000,1.535714,2.0,2.0,2.0
2811,29,1,1,2,2011,0,0,0,1.97,0,...,0,0,1,0,0,0.571429,1.107143,0.0,1.0,3.0
2812,29,2,1,2,2011,0,0,0,1.98,0,...,0,1,0,0,1,2.571429,2.964286,2.0,5.0,4.0
2821,29,4,1,2,2011,0,0,0,12.42,0,...,0,0,1,0,0,0.571429,0.535714,1.0,3.0,0.0
2822,29,0,1,2,2011,0,0,0,7.27,0,...,1,0,0,0,1,0.142857,0.321429,0.0,1.0,1.0


## Machine Learning